# Finetuning T5-sl-small on Oasst1-sl

Code adapted from [Fine-Tune T5 🤗 for Conversational Model](https://www.kaggle.com/code/kreeshrajani/fine-tune-t5-for-conversational-model)

In [1]:
import json
from sklearn.model_selection import train_test_split

import torch
import pandas as pd
import pytorch_lightning as pl
from torch.nn.utils.rnn import pad_sequence
from pytorch_lightning.callbacks import ModelCheckpoint
from transformers import AutoTokenizer, T5ForConditionalGeneration, AdamW

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
INPUT_MAX_LEN = 512 #input length
OUTPUT_MAX_LEN = 512 # output length
TRAIN_BATCH_SIZE = 8 # batch size of training
VAL_BATCH_SIZE = 2 # batch size for validation
EPOCHS = 5 # number of epoch
MODEL_NAME = "cjvt/t5-sl-small"

In [3]:
data_path = '../input/oasst1-sl/google_translate.jsonl'
with open(data_path, 'r') as f:
    data = [json.loads(line) for line in f]

In [4]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)

In [5]:
text = data[0]['prompt']['translation']    # assume the text that is to be tokenized 

input_tokenize = tokenizer( 
             text,
            add_special_tokens=True,        #Add Special tokens like [CLS] and [SEP]
            max_length=INPUT_MAX_LEN,
            padding = 'max_length',         #for padding to max_length for equal sequence length
            truncation = True,              #truncate the text if it is greater than max_length
            return_attention_mask=True,     #will return attention mask
            return_tensors="pt"             #return tensor formate
        )

In [6]:
text, input_tokenize['input_ids'].shape

('Ali imate kakšne informacije o Commodore 64?', torch.Size([1, 512]))

In [7]:
def tree_to_pairs(tree):
    # get pairs for root prompt and replies
    pairs = message_to_pairs(tree["prompt"])
    return pairs

def message_to_pairs(message):
    pairs = []
    # check who is the input role (we want only pairs where the input is the prompter)
    input_text = message['translation']
    # check if we have any replies to build pairs from
    if 'replies' in message:
        for reply in message['replies']:
            # get pairs for current input text ang reply
            if 'translation' not in reply:
                continue
            pairs.append((input_text, reply['translation'], reply['role']))
            # get pairs for reply and its replies
            pairs.extend(message_to_pairs(reply))
    return pairs

In [8]:
pairs = []
for tree in data:
    pairs += tree_to_pairs(tree)
pairs_df = pd.DataFrame(pairs, columns=['question', 'answer', 'reply_role'])

mask = pairs_df['reply_role'] == "assistant"
pairs_df = pairs_df[mask]
pairs_df.drop(columns=['reply_role'], inplace=True)
display(pairs_df)

,question,answer
0,Ali imate kakšne informacije o Commodore 64?,Seveda. Commodore 64 je 8-bitni domači računal...
1,Ali imate kakšne informacije o Commodore 64?,"Seveda, tukaj je kratek povzetek trenutnega čl..."
7,Ali imate kakšne informacije o Commodore 64?,Commodore 64 (C64) je 8-bitni domači računalni...
9,Ali imate kakšne informacije o Commodore 64?,ja Bi še kaj posebej radi vedeli?
16,Metoda DMAIC v akciji\nProduktna skupina opaža...,"Metoda DMAIC (Define, Measure, Analyze, Improv..."
...,...,...
67035,Katere jezike govorijo v Kostariki?,"Primarni jezik, ki se govori v Kostariki, je š..."
67036,Katere jezike govorijo v Kostariki?,Uradni nacionalni jezik Kostarike je španščina...
67037,Kaj je umetna nevronska mreža?,Umetna nevronska mreža (ANN) je računalniški m...
67038,Kaj je umetna nevronska mreža?,"Umetna nevronska mreža je računalniški model, ..."


In [9]:
class T5Dataset:
    
  def __init__(self,question,answer):   
    
    self.question = question
    self.answer = answer
    self.tokenizer = tokenizer
    self.input_max_len = INPUT_MAX_LEN
    self.output_max_len = OUTPUT_MAX_LEN
  
  def __len__(self):                      # This method retrives the number of item from the dataset
    return len(self.question)

  def __getitem__(self,item):             # This method retrieves the item at the specified index item. 

    question = str(self.question[item])
    question = ''.join(question.split())

    answer = str(self.answer[item])
    answer = ''.join(answer.split())

    input_tokenize = self.tokenizer(      
            question,
            add_special_tokens=True,
            max_length=self.input_max_len,
            padding = 'max_length',
            truncation = True,
            return_attention_mask=True,
            return_tensors="pt"
        )
    output_tokenize = self.tokenizer(
            answer,
            add_special_tokens=True,
            max_length=self.output_max_len,
            padding = 'max_length',
            truncation = True,
            return_attention_mask=True,
            return_tensors="pt"
            
        )
    

    input_ids = input_tokenize["input_ids"].flatten()
    attention_mask = input_tokenize["attention_mask"].flatten()
    labels = output_tokenize['input_ids'].flatten()

    out = {
            'question':question,      
            'answer':answer,
            'input_ids': input_ids,
            'attention_mask':attention_mask,
            'target':labels
        }
        
    return out      

In [10]:
class T5DataLoad(pl.LightningDataModule):
    
    def __init__(self,df_train,df_test):
        super().__init__()
        self.df_train = df_train
        self.df_test = df_test
        self.tokenizer = tokenizer
        self.input_max_len = INPUT_MAX_LEN
        self.out_max_len = OUTPUT_MAX_LEN
    
    def setup(self, stage=None):
        
        self.train_data = T5Dataset(
            question = self.df_train.question.values,
            answer = self.df_train.answer.values
        )
        
        self.valid_data = T5Dataset(
            question = self.df_test.question.values,
            answer = self.df_test.answer.values
        )
    def train_dataloader(self):
        return torch.utils.data.DataLoader(
         self.train_data,
         batch_size= TRAIN_BATCH_SIZE,
         shuffle=True, 
         num_workers=2
        )
    def val_dataloader(self):
        return torch.utils.data.DataLoader(
        self.valid_data,
        batch_size= VAL_BATCH_SIZE,
        num_workers = 2
        )


In [11]:
class T5Model(pl.LightningModule):
    
    def __init__(self):
        super().__init__()
        self.model = T5ForConditionalGeneration.from_pretrained(MODEL_NAME, return_dict = True)

        
    def forward(self, input_ids, attention_mask, labels=None):
        
        output = self.model(
        input_ids=input_ids, 
        attention_mask=attention_mask, 
        labels=labels
        )
        return output.loss, output.logits
    
    def training_step(self, batch, batch_idx):

        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels= batch["target"]
        loss, logits = self(input_ids , attention_mask, labels)

        
        self.log("train_loss", loss, prog_bar=True, logger=True)

        return {'loss': loss}
    
    def validation_step(self, batch, batch_idx):
        input_ids = batch["input_ids"]
        attention_mask = batch["attention_mask"]
        labels= batch["target"]
        loss, logits = self(input_ids, attention_mask, labels)

        self.log("val_loss", loss, prog_bar=True, logger=True)
        
        return {'val_loss': loss}

    def configure_optimizers(self):
        return AdamW(self.parameters(), lr=0.0001)

In [12]:
def run():
    df_train, df_test = train_test_split(pairs_df,test_size = 0.2, random_state=100)
    dataload = T5DataLoad(df_train,df_test)
    dataload.setup()
    device = DEVICE
    model = T5Model()
    #model.to(device)
    
    checkpoint = ModelCheckpoint(
        dirpath="/kaggle/working",
        filename='best-model',
        save_top_k=2,
        verbose=True,
        monitor="val_loss",
        mode="min"
    )
    trainer = pl.Trainer(
        callbacks = checkpoint,
        max_epochs= 1,
        devices=2,
        accelerator="auto"
    )
    trainer.fit(model, dataload)

In [13]:
run()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/opt/conda/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory /kaggle/working exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Sanity Checking: 0it [00:00, ?it/s]

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:432: PossibleUserWarning: It is recommended to use `self.log('val_loss', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
  warning_cache.warn(


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

In [15]:
train_model = T5Model.load_from_checkpoint('/kaggle/working/best-model.ckpt')
train_model.freeze()

def generate_question(question):

    inputs_encoding =  tokenizer(
        question,
        add_special_tokens=True,
        max_length= INPUT_MAX_LEN,
        padding = 'max_length',
        truncation='only_first',
        return_attention_mask=True,
        return_tensors="pt"
        )

    
    generate_ids = train_model.model.generate(
        input_ids = inputs_encoding["input_ids"],
        attention_mask = inputs_encoding["attention_mask"],
        max_length = INPUT_MAX_LEN,
        num_beams = 4,
        num_return_sequences = 1,
        no_repeat_ngram_size=2,
        early_stopping=True,
        )

    preds = [
        tokenizer.decode(gen_id,
        skip_special_tokens=True, 
        clean_up_tokenization_spaces=True)
        for gen_id in generate_ids
    ]

    return "".join(preds)

In [25]:
question = pairs_df.iloc[69].question
print("Question: ",question)
print("Assistant: ",generate_question(question))

Question:  Zakaj naslednja koda js natisne niz "banana" na konzoli?
const word = ('b' + 'a' + + 'a' + 'a' ).toLowerCase();
console.log(beseda);
?
Assistant:  ݄:'a';'b');'c'='d'+'*'' +'A'('B')'^'`^^``{'ba'}{`bash(`b',`a','f')){{}`}}#{#${$#}$$;$=$_$($);$||`|'{"a"'s'$`;`'#`#'"##',#""$";#,#.#(#){("b");};}.$},'n':#;{|}|{\{={*}(});{^}=`c`s',}'x'>'<{<}<<#</#><$\$->`<`$<=\`*`**`


In [27]:
def continue_run(checkpoint='/kaggle/working/best-model.ckpt'):
    df_train, df_test = train_test_split(pairs_df,test_size = 0.2, random_state=100)
    dataload = T5DataLoad(df_train,df_test)
    dataload.setup()
    device = DEVICE
    model = T5Model.load_from_checkpoint(checkpoint)
    #model.to(device)
    
    checkpoint = ModelCheckpoint(
        dirpath="/kaggle/working",
        filename='best-model',
        save_top_k=2,
        verbose=True,
        monitor="val_loss",
        mode="min"
    )
    trainer = pl.Trainer(
        callbacks = checkpoint,
        max_epochs= 1,
        devices=2,
        accelerator="auto"
    )
    trainer.fit(model, dataload)

In [ ]:
continue_run()

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


/opt/conda/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:612: UserWarning: Checkpoint directory /kaggle/working exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Sanity Checking: 0it [00:00, ?it/s]

/opt/conda/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/result.py:432: PossibleUserWarning: It is recommended to use `self.log('val_loss', ..., sync_dist=True)` when logging on epoch level in distributed setting to accumulate the metric across devices.
  warning_cache.warn(


Training: 0it [00:00, ?it/s]